# 04. 토치텍스트(TorchText)의 batch_first
---
torchtext에서 batch_first를 True로 한 경우와 False로 한 경우를 비교해보자

### 1. 훈련 데이터와 테스트 데이터로 분리하기
---

In [1]:
import urllib.request
import pandas as pd

In [2]:
exist_data = 'D:/chch/startwhithtorch/datafolder/IMDb_Reviews.csv'
df = pd.read_csv(exist_data, encoding='latin1')

In [3]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [4]:
train_df = df[:25000]
test_df = df[25000:]

In [5]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

### 2. 필드 정의하기(torchtext.data)
---

In [12]:
from torchtext.legacy import data

# text 필드 정의
TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = str.split,
                  lower = True,
                  batch_first = True, # 이 부분을 바꿀건데 이전에 했던 방법처럼 True해서 우선 확인
                                      # batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
                  fix_length = 20)

# label 필드 정의
LABEL = data.Field(sequential=False,
                   use_vocab = False,
                   batch_first = False,
                   is_target = True)

### 3. 데이터셋 / 단어 집합 / 데이터로더 만들기
---

In [16]:
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator

# 데이터 불러오면서 토큰화
tabul_path = 'D:/chch/startwhithtorch/datafolder/'
train_data, test_data = TabularDataset.splits(
    path = tabul_path, train = 'train_data.csv', test = 'test_data.csv', format='csv',
    fields = [('text', TEXT), ('label', LABEL)], skip_header = True
)

# 단어 집합 생성
TEXT.build_vocab(train_data, min_freq = 10, max_size=10000)

# 배치 크기를 정하고 첫번째 배치 출력
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
batch = next(iter(train_loader))

In [20]:
print(batch.text.size())
print(batch.text)

torch.Size([5, 20])
tensor([[  22,   16, 4217,   18,    9,   61,   37,  112,  329, 3471, 1187,   17,
          329,    0,   34,  135,  617, 2386, 1895,    0],
        [   9,   37,    2,   84, 6263,    9,   37,    2,    0,   18,  148,    3,
           20,   12,  271,   37,  147,  268,   29,   11],
        [  10,   20,    7,  300, 5367,   11,    7,   38, 2440,  285,    2,    0,
            4,   80,    0,   23,  257,  804,    2,  274],
        [  52,   23, 1833,  127, 8071,    6,    0,   18,  529,  393,    7,  171,
         8307,  686,    0,    8,   10,  149,   70,   23],
        [ 186,  310,    3,  156, 1633,   42, 7343,    4,    0,    4,   10,    7,
          362,   51, 2050,   68,    2,    0, 2410,    8]])


# 4. 필드 재정의하여 batch_first를 false로 한 경우를 확인하자
---

In [7]:
from torchtext.legacy import data
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator
# ----------------------------------------------
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  batch_first=False,
                  lower=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)
# ----------------------------------------------

# 데이터 불러오면서 토큰화
tabul_path = 'D:/chch/startwhithtorch/datafolder/'
train_data, test_data = TabularDataset.splits(
    path = tabul_path, train = 'train_data.csv', test = 'test_data.csv', format='csv',
    fields = [('text', TEXT), ('label', LABEL)], skip_header = True
)

# 단어 집합 생성
TEXT.build_vocab(train_data, min_freq = 10, max_size=10000)

# 배치 크기를 정하고 첫번째 배치 출력
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
batch = next(iter(train_loader))

In [8]:
print(batch.text.size())
print(batch.text)

torch.Size([20, 5])
tensor([[   0,   10,   10,    0, 5077],
        [  23,  131,   20,    0,    0],
        [2740,    7,  354,  271,    0],
        [  35,   30, 3322,    4,    0],
        [   0,    5,    0,  946, 3619],
        [   2,    2,   21,  854,    0],
        [ 158,  246,  667, 2492,   13],
        [1391,  275,    6,  267, 4562],
        [ 296,    5,   86,   34,   86],
        [  47,   36,    3,    2,   24],
        [   5, 4475,   87,    0,  115],
        [2681,  357,  255, 3929,   50],
        [   6,   60,  150,    9,  291],
        [   0,  238,   18,  235,    8],
        [  33,  762,    9,    8,    2],
        [  62,    4,   61,   11,  769],
        [  59,  434,  152,    3,    5],
        [ 316,  201,    0, 2723,    3],
        [   0,    3,    0,  926, 2753],
        [4482,  403,  122,   12,  659]])


In [ ]:
# 20, 5 로 배치사이즈가 차원의 맨 앞으로 왔다.